## 多线程访问同一个列表导致冲突的问题

两个线程同时往一个 List 中加入`MAX_COUNT`个数，最终 List 的数量有可能并非是`2 * MAX_COUNT`


In [1]:
#load "LoadBaseToolWPF.csx"
using BaseTool;
using System.Threading;
using System.IO;

const int MAX_COUNT = 10;
int _count = 0;
while (_count < MAX_COUNT)
{
    List<int> list = new List<int>();

    Task _task1 = Task.Run(() =>
    {
        for (int i = 0; i < MAX_COUNT; i++) { list.Add(i); }
    });

    Task _task2 = Task.Run(() =>
    {
        for (int i = MAX_COUNT; i < 2 * MAX_COUNT; i++) { list.Add(i); }
    });

    Task.WaitAll(_task1, _task2);

    // 当长度对不上时，输出详细内容
    Console.WriteLine($"{_count++}:数量{list.Count}");
    if (list.Count != MAX_COUNT * 2) { list.Dump(); }
    Thread.Sleep(1000);
}

0:数量20
1:数量20
2:数量20
3:数量20
4:数量20
5:数量20
6:数量20
7:数量20
8:数量20
9:数量20


## 测试直接运行异步函数，而不使用 await 关键字的运行情况


In [2]:
using System.Diagnostics;

Stopwatch _sw = Stopwatch.StartNew();
await Function();
_sw.Stop();
Console.WriteLine($"运行耗时：{_sw.ElapsedMilliseconds}ms");

// 这是一个异步函数，如果不直接执行，则会直接跳过
public async Task Function()
{
    await Task.Delay(2000);
    Console.WriteLine("Hello World");
}

Hello World
运行耗时：2001ms


In [3]:
using System.Diagnostics;

Stopwatch _sw = Stopwatch.StartNew();
Function();
_sw.Stop();
Console.WriteLine($"运行耗时：{_sw.ElapsedMilliseconds}ms");

// 这是一个异步函数，如果不直接执行，则会直接跳过
public async Task Function()
{
    await Task.Delay(2000);
    Console.WriteLine("Hello World");
}

运行耗时：0ms


## 多线程打断等待测试

- 任务 1 持续循环，等待任务 2 打断后，再过 1 秒，结束。
- 任务 2 等待 2 秒，打断任务 1，等待任务 1 完全结束，也结束。

这里有个问题：
如果任务2中没有`await _task`，即取消后就结束任务2，那么任务1最终就不会执行。

In [12]:
using System.Threading.Tasks;

CancellationTokenSource _cts = new CancellationTokenSource();
Task _task = Task.Run(()=>
{
    while(true)
    {
        if(_cts.IsCancellationRequested) break;
    }
    Task.Delay(1000).Wait();
    Console.WriteLine("任务1结束");
});

await Task.Run(async ()=>
{
    Task.Delay(2000).Wait();
    _cts.Cancel();
    await _task;
    Console.WriteLine("任务2结束");
});

任务1结束
任务2结束
